In [146]:
from glob import glob
import os
import joblib
import mne
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [147]:
eeg_channels = ['Fp1',
 'Fp2',
 'F3',
 'F4',
 'F7',
 'F8',
 'T3',
 'T4',
 'T5',
 'T6',
 'Fz']

In [148]:
def read_data_test(file_path):
    data=mne.io.read_raw_eeglab(file_path,preload=True)
    data.set_eeg_reference()
    data = data.pick_channels(eeg_channels)
    #data.filter(l_freq=0.5,h_freq=45)
    epochs=mne.make_fixed_length_epochs(data,duration=5,overlap=1)
    array=epochs.get_data()
    return array

In [149]:
from scipy import stats

def mean(data):
    return np.mean(data,axis=-1)
    
def std(data):
    return np.std(data,axis=-1)

def ptp(data):
    return np.ptp(data,axis=-1)

def var(data):
        return np.var(data,axis=-1)

def minim(data):
      return np.min(data,axis=-1)


def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)


def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): #root mean square
      return  np.sqrt(np.mean(data**2,axis=-1))  

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

def skewness(data):
    return stats.skew(data,axis=-1)

def kurtosis(data):
    return stats.kurtosis(data,axis=-1)

def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),var(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kurtosis(data)),axis=-1)

In [150]:
test_dict={0:'Alzheimers',1:'Dementia'}

In [166]:
test_file_path=glob('derivatives/F/*.set')
print(len(test_file_path))

23


In [223]:
%%capture
sample_data_pred=read_data_test(test_file_path[3])

In [224]:
from tqdm import tqdm_notebook
featuress=[]
for data in tqdm_notebook(sample_data_pred):
    featuress.append(concatenate_features(data))
featuress=np.array(featuress)
featuress.shape

C:\Users\91904\AppData\Local\Temp\ipykernel_19276\2936894702.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for data in tqdm_notebook(sample_data_pred):


  0%|          | 0/158 [00:00<?, ?it/s]

(158, 143)

In [225]:
loaded_model = joblib.load('models/lda_AF.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Dementia'

In [226]:
loaded_model = joblib.load('models/LR_AF.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Alzheimers'

In [227]:
loaded_model = joblib.load('models/LDA_Kfold_AF.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Dementia'

In [228]:
loaded_model = joblib.load('models/XGboost_AF.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Dementia'

In [229]:
loaded_model = joblib.load('models/RF_Kfold_AF.sav')
test_dict.get(np.bincount(loaded_model.predict(featuress)).argmax())

'Dementia'